In [18]:

from network_evaluation_tools import data_import_tools as dit
from network_evaluation_tools import network_evaluation_functions as nef
from network_evaluation_tools import network_propagation as prop
import pandas as pd
import numpy as np

In [5]:
cd ..

/home/karen/Documents/GitHub/Network_Evaluation_Tools


In [1]:
evaluation_network_folder = 'Data/Networks/'
evaluation_lists_folder = 'Data/Evaluations/'

In [20]:
from os import listdir, path
from os.path import isfile, join


In [13]:
list_networks = [evaluation_network_folder+f for f in listdir(evaluation_network_folder) if isfile(join(evaluation_network_folder, f))]

In [16]:
list_evaluations

['Data/Evaluations/DisGeNET_genesets.txt',
 'Data/Evaluations/tissues.txt',
 'Data/Evaluations/systems.txt']

In [15]:
list_evaluations = [evaluation_lists_folder+f for f in listdir(evaluation_lists_folder) if isfile(join(evaluation_lists_folder, f))]

In [33]:
network_name = path.splitext(evaluating_file)[0].split(evaluation_lists_folder)[1] +"_" + path.splitext(evaluating_network)[0].split(evaluation_network_folder)[1]

In [34]:
network_name 

'DisGeNET_genesets_Young2Old_95_edge_14'

In [37]:
%%capture
performances = {}
gains = {}
for evaluating_file in list_evaluations:
    genesets = dit.load_node_sets(evaluating_file)
    for evaluating_network in list_networks:
        network_name = path.splitext(evaluating_file)[0].split(evaluation_lists_folder)[1] +"_" + path.splitext(evaluating_network)[0].split(evaluation_network_folder)[1]
        genesets_p = nef.calculate_p(network, genesets)
        network_nodes = [str(gene) for gene in network.nodes()]
        nodesets_p = {}
        selected = {}
        m=-0.18887257
        b=0.64897403
        nodesets = genesets
        for nodeset in nodesets:
                nodesets_coverage = len([node for node in nodesets[nodeset] if node in network_nodes])
                if nodesets_coverage > 0:
                    nodesets_p[nodeset] = round(m*np.log10(nodesets_coverage)+b, 4)
                    selected[nodeset] = nodesets[nodeset]
                else:
                    print nodeset, ' does not have common genes'
        genesets_p = nodesets_p
        genesets = selected
        alpha = prop.calculate_alpha(network)
        kernel = nef.construct_prop_kernel(network, alpha=alpha, verbose=True)
        AUPRC_values = nef.small_network_AUPRC_wrapper(kernel, genesets, genesets_p, n=30, cores=1, verbose=True)
        null_AUPRCs = []
        for i in range(10):
            shuffNet = nef.shuffle_network(network, max_tries_n=10, verbose=True)
            shuffNet_kernel = nef.construct_prop_kernel(shuffNet, alpha=alpha, verbose=False)
            shuffNet_AUPRCs = nef.small_network_AUPRC_wrapper(shuffNet_kernel, genesets, genesets_p, n=30, cores=4, verbose=False)
            null_AUPRCs.append(shuffNet_AUPRCs)
            print 'shuffNet', repr(i+1), 'AUPRCs calculated'
        null_AUPRCs_table = pd.concat(null_AUPRCs, axis=1)
        null_AUPRCs_table.columns = ['shuffNet'+repr(i+1) for i in range(len(null_AUPRCs))]
        network_performance = nef.calculate_network_performance_score(AUPRC_values, null_AUPRCs_table, verbose=True)
        network_performance.name = network_name
        network_perf_gain = nef.calculate_network_performance_gain(AUPRC_values, null_AUPRCs_table, verbose=True)
        network_perf_gain.name = network_name
        performances[network_name]=network_performance
        gains[network_name]=network_performance

        

In [67]:
performances['elements']= section_lens
gains['elements']= section_lens

In [68]:
df_p = pd.DataFrame(performances)
df_p.to_csv('Data/Network_Performance_J.csv')

In [69]:
df_g = pd.DataFrame(performances)
df_g.to_csv('Data/Network_Performance_Gain_J.csv')

In [71]:
pd.Series(performances['DisGeNET_genesets_Young2Old_95_edge_14']).plot(kind='hist', bins=[0,10,20]) # `bins` defines the start and end points of bins


In [21]:
# Rank network on average performance across gene sets vs performance on same gene sets in previous network set
all_network_performance = pd.read_csv('Data/Network_Performance_J.csv', index_col=0, header=None)


In [22]:
all_network_performance_filt = pd.concat([network_performance, all_network_performance.ix[network_performance.index]], axis=1)
network_performance_rank_table = all_network_performance_filt.rank(axis=1, ascending=False)
network_performance_rankings = network_performance_rank_table['Test Network']

/home/karen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [23]:
# Rank network on average performance gain across gene sets vs performance gain on same gene sets in previous network set
all_network_perf_gain = pd.read_csv('Data/Network_Performance_Gain_J.csv', index_col=0, header=None)
all_network_perf_gain_filt = pd.concat([network_perf_gain, all_network_perf_gain.ix[network_perf_gain.index]], axis=1)



/home/karen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
all_network_perf_gain_filt[:6]

,Test Network,1
brain,-0.000000,-0.000000
kidney,-0.000000,-0.000000
pancreas,-0.000000,-0.000000
skin,-0.000000,-0.000000
spleen,-0.000000,-0.000000
testis,0.019718,0.019718


In [25]:
network_perf_gain_rank_table = all_network_performance_filt.rank(axis=1, ascending=False)



In [26]:
network_perf_gain_rankings = network_perf_gain_rank_table['Test Network']

In [27]:
network_perf_gain_rank_table

,Test Network,1
brain,NaN,NaN
kidney,NaN,NaN
pancreas,NaN,NaN
skin,NaN,NaN
spleen,NaN,NaN
testis,1.5,1.5


In [28]:
# Network Performance
network_performance_metric_ranks = pd.concat([network_performance, network_performance_rankings, network_perf_gain, network_perf_gain_rankings], axis=1)
network_performance_metric_ranks.columns = ['Network Performance', 'Network Performance Rank', 'Network Performance Gain', 'Network Performance Gain Rank']
network_performance_metric_ranks.sort_values(by=['Network Performance Rank', 'Network Performance', 'Network Performance Gain Rank', 'Network Performance Gain'],
                                             ascending=[True, False, True, False])

,Network Performance,Network Performance Rank,Network Performance Gain,Network Performance Gain Rank
testis,0.666233,1.5,0.019718,1.5
brain,NaN,NaN,-0.000000,NaN
kidney,NaN,NaN,-0.000000,NaN
pancreas,NaN,NaN,-0.000000,NaN
skin,NaN,NaN,-0.000000,NaN
spleen,NaN,NaN,-0.000000,NaN


In [29]:
# Construct network summary table
network_summary = {}
network_summary['Nodes'] = int(len(network.nodes()))
network_summary['Edges'] = int(len(network.edges()))


In [30]:
network_summary['Avg Node Degree'] = np.mean(dict(network.degree()).values())


In [31]:
network.degree()

DegreeView({u'HDAC1': 1, u'TGFB1': 2, u'BAG6': 1, u'EBNA-LP': 1, u'CRELD2': 1, u'EEF1A1': 1, u'EEF1A2': 18, u'COPS6': 1, u'COPS5': 2, u'Source': 1, u'CDKN2A': 28, u'hsa-miR-10b-5p': 2, u'GMNN': 1, u'OTUB1': 1, u'hsa-miR-181a-5p': 2, u'CDC6': 1, u'CDC7': 2, u'ubiq_human': 1, u'hsa-miR-22-3p': 2, u'LRRK2': 3, u'Terget': 1, u'TUBA1A': 1, u'DEAF1': 1, u'CKS1B': 1, u'hsa-miR-519d-3p': 1, u'hsa-miR-205-5p': 2, u'CDK3': 1, u'MAPK6': 2, u'hsa-miR-92a-3p': 2, u'KSR1': 1, u'hsa-miR-93-5p': 1, u'HRAS': 2, u'hsa-miR-29a-3p': 3, u'PCNA': 1, u'hsa-miR-21-5p': 2, u'MCM10': 1, u'PTEN': 15, u'RAB1A': 1, u'SP1': 5, u'M': 1, u'BMI1': 2, u'EEF1B2': 1, u'P': 1, u'hsa-miR-519a-3p': 1, u'hsa-miR-17-5p': 3, u'MDM2': 2, u'hsa-miR-29b-3p': 4, u'hsa-miR-200c-3p': 3, u'TP53': 2, u'CCND1': 3, u'hsa-miR-106a-5p': 1, u'CUL4A': 1, u'hsa-miR-155-5p': 2, u'HTT': 1, u'CDK4': 1, u'FGB': 1, u'CDK6': 3, u'CDC45': 1, u'ACTB': 1, u'PSEN1': 1})

In [32]:
network_summary['Edge Density'] = 2*network_summary['Edges'] / float((network_summary['Nodes']*(network_summary['Nodes']-1)))
network_summary['Avg Network Performance Rank'] = network_performance_rankings.mean()
network_summary['Avg Network Performance Rank, Rank'] = int(network_performance_rank_table.mean().rank().ix['Test Network'])
network_summary['Avg Network Performance Gain Rank'] = network_perf_gain_rankings.mean()
network_summary['Avg Network Performance Gain Rank, Rank'] = int(network_perf_gain_rank_table.mean().rank().ix['Test Network'])
for item in ['Nodes', 'Edges' ,'Avg Node Degree', 'Edge Density', 'Avg Network Performance Rank', 'Avg Network Performance Rank, Rank',
             'Avg Network Performance Gain Rank', 'Avg Network Performance Gain Rank, Rank']:
    print item+':\t'+repr(network_summary[item])

Nodes:	60
Edges:	76
Avg Node Degree:	2.533333333333333
Edge Density:	0.04293785310734463
Avg Network Performance Rank:	1.5
Avg Network Performance Rank, Rank:	1
Avg Network Performance Gain Rank:	1.5
Avg Network Performance Gain Rank, Rank:	1


/home/karen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/karen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
